# Human Detection using HOG Feature and SVM Classifier

- Algorithm Detail : https://learnopencv.com/histogram-of-oriented-gradients/
- Lecture : http://www.cs.utoronto.ca/~fidler/slides/CSC420/lecture17.pdf

In [ ]:
from skimage.feature import hog
#from skimage.io import imread
import joblib,glob,os,cv2

from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
files.upload()

KeyboardInterrupt: ignored

In [ ]:
! unzip /content/DATAIMAGE.zip

### HOG Features

In [ ]:
def feature_extract(img):
  fd = cv2.resize(img,(64,128))
  fd,hog_image = hog(fd,orientations=9,pixels_per_cell=(8,8),visualize=True,cells_per_block=(3,3))
  return hog_image

In [ ]:
test_image = "/content/DATAIMAGE/positive/crop001001d.png"
img = cv2.imread(test_image,0)
plt.imshow(img,cmap='gray')

In [ ]:
result = feature_extract(img)
plt.imshow(result,cmap='gray')

### Data Preparation

In [ ]:
X = []
Y = []

pos_im_path = '/content/DATAIMAGE/positive'
neg_im_path = '/content/DATAIMAGE/negative'

In [ ]:
# Load the positive features
for filename in glob.glob(os.path.join(pos_im_path,"*.png")):
    fd = cv2.imread(filename,0)
    fd = cv2.resize(fd,(64,128))
    fd = hog(fd,orientations=9,pixels_per_cell=(8,8),visualize=False,cells_per_block=(3,3))
    X.append(fd)
    Y.append(1)

In [ ]:
# Load the negative features
for filename in glob.glob(os.path.join(neg_im_path,"*.jpg")):
    fd = cv2.imread(filename,0)
    fd = cv2.resize(fd,(64,128))
    fd = hog(fd,orientations=9,pixels_per_cell=(8,8),visualize=False,cells_per_block=(3,3))
    X.append(fd)
    Y.append(0)

In [ ]:
X = np.float32(X)
Y = np.array(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
print('Train Data:',len(X_train))
print('Train Labels (1,0)',len(y_train))

### Model Training

In [ ]:
model = LinearSVC()
model.fit(X_train,y_train)

In [ ]:
# predict
y_pred = model.predict(X_test)

In [ ]:
# confusion matrix and accuracy

from sklearn import metrics
from sklearn.metrics import classification_report

print(f"Classification report for classifier {model}:\n"
      f"{metrics.classification_report(y_test, y_pred)}\n")

### Demo

In [ ]:
joblib.dump(model, 'models.dat')
print('Model saved : {}'.format('models.dat'))

In [ ]:
import numpy as np
import cv2,joblib
from imutils.object_detection import non_max_suppression
import imutils
from skimage.feature import hog
from skimage import color
from skimage.transform import pyramid_gaussian

filename = "/content/burmese-monks.jpg"

image = cv2.imread(filename)
image = cv2.resize(image,(400,256))
size = (64,128)
step_size = (9,9)
downscale = 1.25
#List to store the detections
detections = []
#The current scale of the image
scale = 0
model = joblib.load('models.dat')

def sliding_window(image, window_size, step_size):
    for y in range(0, image.shape[0], step_size[1]):
        for x in range(0, image.shape[1], step_size[0]):
            yield (x, y, image[y: y + window_size[1], x: x + window_size[0]])

for im_scaled in pyramid_gaussian(image, downscale = downscale):
    #The list contains detections at the current scale
    if im_scaled.shape[0] < size[1] or im_scaled.shape[1] < size[0]:
        break
    for (x, y, window) in sliding_window(im_scaled, size, step_size):
        if window.shape[0] != size[1] or window.shape[1] != size[0]:
            continue
        window = color.rgb2gray(window)

        fd=hog(window, orientations=9,pixels_per_cell=(8,8),visualize=False,cells_per_block=(3,3))
        fd = fd.reshape(1, -1)
        pred = model.predict(fd)
        if pred == 1:

            if model.decision_function(fd) > 0.5:
                detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), model.decision_function(fd),
                int(size[0] * (downscale**scale)),
                int(size[1] * (downscale**scale))))

    scale += 1
clone = image.copy()
clone = cv2.cvtColor(clone,cv2.COLOR_BGR2RGB)
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections])
sc = [score[0] for (x, y, score, w, h) in detections]
print ("sc: ", sc)
sc = np.array(sc)
pick = non_max_suppression(rects, probs = sc, overlapThresh = 0.5)
for(x1, y1, x2, y2) in pick:
    cv2.rectangle(clone, (x1, y1), (x2, y2), (0, 255, 0))
    cv2.putText(clone,'Human',(x1-2,y1-2),1,0.75,(255,255,0),1)

plt.imshow(clone)

In [ ]:
import io
import pandas as pd
pd.read_csv(io.StringIO('''
2684489465_32ba1d0344.jpg
2684489465_32ba1d0344.jpg
2684489465_32ba1d0344.jpg
2684489465_32ba1d0344.jpg
2684489465_32ba1d0344.jpg
2685139184_4ff45e0f76.jpg
2685139184_4ff45e0f76.jpg
2685139184_4ff45e0f76.jpg
2685139184_4ff45e0f76.jpg
2685139184_4ff45e0f76.jpg
2685752892_9d5cd7f274.jpg
2685752892_9d5cd7f274.jpg
2685752892_9d5cd7f274.jpg
2685752892_9d5cd7f274.jpg
2685752892_9d5cd7f274.jpg
'''), header=None)


In [ ]:
import io
import pandas as pd
pd.read_csv(io.StringIO('''
2684489465_32ba1d0344.jpg
2684489465_32ba1d0344.jpg
2684489465_32ba1d0344.jpg
2684489465_32ba1d0344.jpg
2684489465_32ba1d0344.jpg
2685139184_4ff45e0f76.jpg
2685139184_4ff45e0f76.jpg
2685139184_4ff45e0f76.jpg
2685139184_4ff45e0f76.jpg
2685139184_4ff45e0f76.jpg
2685752892_9d5cd7f274.jpg
2685752892_9d5cd7f274.jpg
2685752892_9d5cd7f274.jpg
2685752892_9d5cd7f274.jpg
2685752892_9d5cd7f274.jpg
2685788323_ceab14534a.jpg
2685788323_ceab14534a.jpg
2685788323_ceab14534a.jpg
2685788323_ceab14534a.jpg
2685788323_ceab14534a.jpg
2686432878_0697dbc048.jpg
2686432878_0697dbc048.jpg
2686432878_0697dbc048.jpg
2686432878_0697dbc048.jpg
2686432878_0697dbc048.jpg
268654674_d29e00b3d0.jpg
268654674_d29e00b3d0.jpg
268654674_d29e00b3d0.jpg
268654674_d29e00b3d0.jpg
268654674_d29e00b3d0.jpg
268704620_8a8cef4cb3.jpg
268704620_8a8cef4cb3.jpg
268704620_8a8cef4cb3.jpg
268704620_8a8cef4cb3.jpg
268704620_8a8cef4cb3.jpg
2687229880_97cfd8148e.jpg
2687229880_97cfd8148e.jpg
2687229880_97cfd8148e.jpg
2687229880_97cfd8148e.jpg
2687229880_97cfd8148e.jpg
2687328779_b4356dab16.jpg
2687328779_b4356dab16.jpg
2687328779_b4356dab16.jpg
2687328779_b4356dab16.jpg
2687328779_b4356dab16.jpg
2687529141_edee32649e.jpg
2687529141_edee32649e.jpg
2687529141_edee32649e.jpg
2687529141_edee32649e.jpg
2687529141_edee32649e.jpg
2687539673_d54a8dc613.jpg
2687539673_d54a8dc613.jpg
2687539673_d54a8dc613.jpg
2687539673_d54a8dc613.jpg
2687539673_d54a8dc613.jpg
2687672606_275169c35d.jpg
2687672606_275169c35d.jpg
2687672606_275169c35d.jpg
2687672606_275169c35d.jpg
2687672606_275169c35d.jpg
2688102742_885e578a3f.jpg
2688102742_885e578a3f.jpg
2688102742_885e578a3f.jpg
2688102742_885e578a3f.jpg
2688102742_885e578a3f.jpg
2688902319_52ceaf4a2a.jpg
2688902319_52ceaf4a2a.jpg
2688902319_52ceaf4a2a.jpg
2688902319_52ceaf4a2a.jpg
2688902319_52ceaf4a2a.jpg
2689001252_e0016c89f0.jpg
2689001252_e0016c89f0.jpg
2689001252_e0016c89f0.jpg
2689001252_e0016c89f0.jpg
2689001252_e0016c89f0.jpg
2689163361_4939875be5.jpg
2689163361_4939875be5.jpg
2689163361_4939875be5.jpg
2689163361_4939875be5.jpg
2689163361_4939875be5.jpg
2689358407_9932f1b20c.jpg
2689358407_9932f1b20c.jpg
2689358407_9932f1b20c.jpg
2689358407_9932f1b20c.jpg
2689358407_9932f1b20c.jpg
2689491604_d8760f57b4.jpg
2689491604_d8760f57b4.jpg
2689491604_d8760f57b4.jpg
2689491604_d8760f57b4.jpg
2689491604_d8760f57b4.jpg
2690538407_7ca157be85.jpg
2690538407_7ca157be85.jpg
2690538407_7ca157be85.jpg
2690538407_7ca157be85.jpg
2690538407_7ca157be85.jpg
2690702549_cf81da8cf6.jpg
2690702549_cf81da8cf6.jpg
2690702549_cf81da8cf6.jpg
2690702549_cf81da8cf6.jpg
2690702549_cf81da8cf6.jpg
'''), header=None)
